<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [514]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import layers
from google.colab import files
!pip install talos
import talos
import datetime as dt
from sklearn.metrics import roc_auc_score
from keras import optimizers 

In [515]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df = train_df.drop(['keyword', 'location','id'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [516]:
train_df.shape

(7613, 2)

In [517]:
x_train = train_df['text']
y_train = train_df['target']

In [518]:
def modelardo(x_train,y_train,x_test,y_test,params):
  vectorizer = CountVectorizer()
  vectorizer.fit(x_train)

  x_train = vectorizer.transform(x_train)
  x_test = vectorizer.transform(x_test)
  input_dim = x_train.shape[1]
  model = Sequential()
  model.add(layers.Dense(params['first neuron units'],input_dim=input_dim,activation=params['first neuron activation']))
  model.add(layers.Dense(1,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = model.fit(x_train,y_train,epochs = params['epochs'],verbose = 0,validation_data = (x_test,y_test),batch_size=params['batch size'])
  return out,model

In [519]:
p = {
    'first neuron units': (2,20,1),
    'first neuron activation': ['relu','elu','sigmoid','softmax','tanh','exponential'],
    'epochs': [1,2,3,4,5,6,7,10,20,100],
    'batch size': (5,100,5)
}

In [521]:
# Búsqueda de hiperparámetros con talos.

x_train.shape
#t = talos.Scan(x_train,y_train,params=p,model=modelardo,experiment_name='tuning',time_limit=(dt.datetime.now() + dt.timedelta(minutes = 10)).strftime("%Y-%m-%d %H:%M"),random_method = 'quantum')
#Este proceso tarda 1,5hs aprox en colab. Y tiene muy poquitos hiperparámetros para tunear

(7613,)

In [523]:
#experiment_data = t.data
#experiment_data.head()

In [524]:
#maxim=experiment_data['val_accuracy'].idxmax()

In [525]:
#experiment_data.iloc[maxim]

In [526]:
# Se vectoriza el texto.

vectorizer = CountVectorizer()
vectorizer.fit(train_df['text'])

data_train = vectorizer.fit_transform(train_df['text']).toarray()


In [527]:
# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

x_train, x_validation, y_train, y_validation = train_test_split(data_train,train_df['target'],random_state=7,test_size = 0.20)


input_dim = x_train.shape[1]

best_model = Sequential()
best_model.add(layers.Dense(2,input_dim=input_dim,activation='elu'))
best_model.add(layers.Dense(1,activation = 'sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
out = best_model.fit(x_train,y_train,epochs = 5,verbose = 0,validation_data = (x_validation,y_validation),batch_size=50)

loss, accuracy = best_model.evaluate(x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = best_model.evaluate(x_validation, y_validation, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9039
Testing Accuracy:  0.8181


In [528]:
prediction0=best_model.predict_classes(x_validation, batch_size=50)
score = roc_auc_score(y_validation, prediction0)
score

0.8066710878589148

In [529]:
#Reentrena con el set completo (data_train + data_validation) y realiza predicción (data_test).

out = best_model.fit(data_train,train_df['target'],epochs = 5,verbose = 1,batch_size=50)
data_test = vectorizer.transform(test_df['text']).toarray()
prediction=best_model.predict_classes(data_test, batch_size=50)
prediction

Epoch 1/5
7613/7613 [==============================] - 3s 369us/step - loss: 0.3414 - accuracy: 0.8860
Epoch 2/5
7613/7613 [==============================] - 3s 330us/step - loss: 0.2937 - accuracy: 0.9017
Epoch 3/5
7613/7613 [==============================] - 3s 349us/step - loss: 0.2552 - accuracy: 0.9166
Epoch 4/5
7613/7613 [==============================] - 3s 361us/step - loss: 0.2229 - accuracy: 0.9320
Epoch 5/5
7613/7613 [==============================] - 3s 334us/step - loss: 0.1958 - accuracy: 0.9417


array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [530]:
# Genera archivo submit.

test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')